In [21]:
import string
import os
import numpy as np
import tensorflow as tf
from tensorflow.keras.applications.xception import Xception
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.utils import to_categorical
from PIL import Image
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input,Embedding,LSTM,Dense,Dropout
#from keras.layers.merge import add
import pickle


ModuleNotFoundError: No module named 'tensorflow.keras.layers.merge'

In [3]:
path ='D:\\image caption'
tokens = 'text\\caption.token.txt'
fileName = path +'\\' +tokens

def readFile(path):
    file = open(path,'r')
    info = file.read()
    file.close()
    
    return info
def analysis(path):
    
    
    data = readFile(fileName)
    data =data.split('\n')
    
    
    discriptions={}
    for record in data:
        img, caption = record.split('\t')
        if img[:-2] in discriptions :
            discriptions[img[:-2]].append(caption)
        else:
            discriptions[img[:-2]] = [caption]
    
    return discriptions


data =analysis(fileName)
data

{'1000268201_693b08cb0e.jpg': ['A child in a pink dress is climbing up a set of stairs in an entry way .',
  'A girl going into a wooden building .',
  'A little girl climbing into a wooden playhouse .',
  'A little girl climbing the stairs to her playhouse .',
  'A little girl in a pink dress going into a wooden cabin .'],
 '1001773457_577c3a7d70.jpg': ['A black dog and a spotted dog are fighting',
  'A black dog and a tri-colored dog playing with each other on the road .',
  'A black dog and a white dog with brown spots are staring at each other in the street .',
  'Two dogs of different breeds looking at each other on the road .',
  'Two dogs on pavement moving toward each other .'],
 '1002674143_1b742ab4b8.jpg': ['A little girl covered in paint sits in front of a painted rainbow with her hands in a bowl .',
  'A little girl is sitting in front of a large painted rainbow .',
  'A small girl in the grass plays with fingerpaints in front of a white canvas with a rainbow on it .',
  'T

In [4]:
 caption = data['1001773457_577c3a7d70.jpg'][1]
def preprocessing(caption):
    caption = caption.replace('-','')
    caption_list=caption.split()
    caption_list =[word.lower() for word in caption_list if (len(word)>1) and (word.isalpha()) ]
    return ' '.join (caption_list)

def clean_text(data):
    for img,captions in data.items():
        for indx, caption in enumerate(captions): 
            data[img][indx]=preprocessing(caption)
    return data 


cleaned_data =clean_text(data)
cleaned_data['1000268201_693b08cb0e.jpg']

['child in pink dress is climbing up set of stairs in an entry way',
 'girl going into wooden building',
 'little girl climbing into wooden playhouse',
 'little girl climbing the stairs to her playhouse',
 'little girl in pink dress going into wooden cabin']

In [5]:
def generate_repository(data):
    repository_vocab =set()
    for img in data.keys():
        [repository_vocab.update(caption.split()) for caption in data[img]]

    return repository_vocab

rep_vocab= generate_repository(cleaned_data)


def write_file(path,data):
    lines=[]
    for img ,captions in data.items():
        for caption in captions:
            lines.append(img +'\t'+ caption)
    lines='\n'.join(lines)


    file=open(path,'w')
    file.write(lines)
    file.close()

write_file('D:\\image caption\\text\\cleaned_tokens.txt',cleaned_data)

In [ ]:
path_images ='D:\\image caption\\images'
model = Xception(include_top = False, pooling = 'avg' )


def feature_extraction(path,model):
    features ={}
    for imageName in os.listdir(path):
        completePath = path_images+'\\'+imageName
        #imagefile =path_images +'\\'+ os.listdir(path_images)[0]
        img = Image.open(completePath)
        img = img.resize((299,299))
        img = np.expand_dims(img,axis=0)
        img = img/127.5
        img = img -1.0
        feature = model.predict(img)
        features[imageName] = feature
        print(feature)
    return features

feature_extraction(path_images,model)

# the values should be the min = -1 and the maximum = 1


1/1 [==============================] - 2s 2s/step
[[0.47340956 0.01730908 0.0733423  ... 0.08557966 0.02102294 0.2376557 ]]
1/1 [==============================] - 0s 189ms/step
[[0.00158201 0.11114132 0.00037429 ... 0.26504397 0.35281494 0.05871565]]
1/1 [==============================] - 0s 188ms/step
[[0.         0.02493168 0.01557382 ... 0.         0.         0.10194097]]
1/1 [==============================] - 0s 188ms/step
[[0.         0.12444524 0.73915887 ... 0.00390425 0.00997239 0.5017138 ]]
1/1 [==============================] - 0s 197ms/step
[[0.04138513 0.         0.01274433 ... 0.00944683 0.6420155  0.04793234]]
1/1 [==============================] - 0s 206ms/step
[[0.         0.         0.0263576  ... 0.3052605  0.23072201 0.14181006]]
1/1 [==============================] - 0s 187ms/step
[[0.2093522  0.         0.09285209 ... 0.10372841 0.04261005 0.        ]]
1/1 [==============================] - 0s 231ms/step
[[0.         0.12888514 0.11070187 ... 0.0012872  0.         

IOPub message rate exceeded.
The Jupyter server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--ServerApp.iopub_msg_rate_limit`.

Current values:
ServerApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
ServerApp.rate_limit_window=3.0 (secs)



[[0.33256978 0.2649653  0.00556547 ... 0.05098531 0.0100005  0.00336605]]
1/1 [==============================] - 0s 220ms/step
[[0.0829637  0.         0.00047882 ... 0.08573548 0.         0.03241069]]
1/1 [==============================] - 0s 192ms/step
[[0.         0.00356046 0.09014366 ... 0.00664675 0.         0.00095083]]
1/1 [==============================] - 0s 188ms/step
[[0.         0.         0.2761252  ... 0.34317642 0.10991897 0.        ]]
1/1 [==============================] - 0s 204ms/step
[[0.03012762 0.0059879  0.10288952 ... 0.08314152 0.         0.        ]]
1/1 [==============================] - 0s 194ms/step
[[0.03523154 0.11493254 0.         ... 0.2897876  0.15142836 0.01139719]]
1/1 [==============================] - 0s 207ms/step
[[0.00126319 0.00613909 0.68607634 ... 0.2992643  0.16696855 0.22626112]]
1/1 [==============================] - 0s 208ms/step
[[0.         0.         0.         ... 0.07363457 0.         0.06838244]]
1/1 [==============================] 

In [7]:
features = feature_extraction(path_images,model)
pickle.dump(features,open('D:\\image caption\\images_features.bin','wb'))

[[0.04116954 0.         0.         ... 0.00119429 0.12672508 0.        ]]
1/1 [==============================] - 0s 206ms/step
[[0.2779692  0.03813374 0.         ... 0.00029694 0.         0.07764263]]
1/1 [==============================] - 0s 195ms/step
[[0.26668727 0.00129212 0.3280765  ... 0.         0.01865952 0.20590691]]
1/1 [==============================] - 0s 193ms/step
[[0.0579708  0.02635623 0.00301593 ... 0.9221847  0.         0.        ]]
1/1 [==============================] - 0s 202ms/step
[[0.0000000e+00 2.1954597e-04 8.7532243e-03 ... 7.3657739e-01
  0.0000000e+00 2.4729467e-03]]
1/1 [==============================] - 0s 191ms/step
[[0.73789996 0.4470417  0.00429456 ... 0.28791225 0.09210127 0.0490412 ]]
1/1 [==============================] - 0s 194ms/step
[[0.         0.         0.01775011 ... 0.0090953  0.3024932  0.07436733]]
1/1 [==============================] - 0s 192ms/step
[[0.         0.04931038 0.05386901 ... 0.8789137  0.         0.04928815]]
1/1 [============

In [8]:
features = pickle.load(open('D:\\image caption\\images_features.bin','rb'))
features

{'1000268201_693b08cb0e.jpg': array([[0.47340956, 0.01730908, 0.0733423 , ..., 0.08557966, 0.02102294,
         0.2376557 ]], dtype=float32),
 '1001773457_577c3a7d70.jpg': array([[0.00158201, 0.11114132, 0.00037429, ..., 0.26504397, 0.35281494,
         0.05871565]], dtype=float32),
 '1002674143_1b742ab4b8.jpg': array([[0.        , 0.02493168, 0.01557382, ..., 0.        , 0.        ,
         0.10194097]], dtype=float32),
 '1007129816_e794419615.jpg': array([[0.        , 0.12444524, 0.73915887, ..., 0.00390425, 0.00997239,
         0.5017138 ]], dtype=float32),
 '1007320043_627395c3d8.jpg': array([[0.04138513, 0.        , 0.01274433, ..., 0.00944683, 0.6420155 ,
         0.04793234]], dtype=float32),
 '1009434119_febe49276a.jpg': array([[0.        , 0.        , 0.0263576 , ..., 0.3052605 , 0.23072201,
         0.14181006]], dtype=float32),
 '1012212859_01547e3f17.jpg': array([[0.2093522 , 0.        , 0.09285209, ..., 0.10372841, 0.04261005,
         0.        ]], dtype=float32),
 '1015

In [9]:
train_images_path = 'D:\\image caption\\text\\caption.trainImages.txt'
clean_tokens_path = 'D:\\image caption\\text\\cleaned_tokens.txt'
features_path = 'D:\\image caption\\images_features.bin'
def load_images(path):
    data = readFile(path)
    data = data.split('\n')

    return data


def load_tokens(path,images):
    data = readFile(path)
    lines= data.split('\n')

    tokens={}
    for line in lines:
        image,caption =line.split('\t')
        if image in images:
            if image not in tokens:
                tokens[image] = []

            tokens[image].append('<start >'+ caption + '<end>')

    return tokens

def load_features(path,images):
    features = pickle.load(open(path,'rb'))

    selected_features = [{image:features[image]} for image in images if image in features]

    return selected_features
    

train_image = load_images(train_images_path)
train_tokens = load_tokens(clean_tokens_path,train_image)
train_features = load_features(features_path,train_image)



In [10]:
train_features

[{'2513260012_03d33305cf.jpg': array([[0.        , 0.44815886, 0.        , ..., 0.17757446, 0.00191513,
          0.00869672]], dtype=float32)},
 {'2903617548_d3e38d7f88.jpg': array([[0.17660895, 0.01287666, 0.15669051, ..., 0.07796577, 0.22157356,
          0.07919015]], dtype=float32)},
 {'3338291921_fe7ae0c8f8.jpg': array([[0.28786096, 0.        , 0.        , ..., 0.        , 0.08850855,
          0.        ]], dtype=float32)},
 {'2644326817_8f45080b87.jpg': array([[0.00466417, 0.00331445, 0.02121833, ..., 0.07942958, 0.01354535,
          0.        ]], dtype=float32)},
 {'218342358_1755a9cce1.jpg': array([[0.07323228, 0.01431645, 0.11561768, ..., 0.11935582, 0.07156453,
          0.11425362]], dtype=float32)},
 {'2501968935_02f2cd8079.jpg': array([[1.1065808e-01, 1.7159581e-04, 3.4686658e-01, ..., 1.2568930e-02,
          1.0891147e-01, 3.8788685e-01]], dtype=float32)},
 {'2699342860_5288e203ea.jpg': array([[0.        , 0.15521923, 0.        , ..., 0.04902746, 0.43784463,
         

In [11]:
# fetch captions on specific pictures 
def fetch_captions(data):
    captions = []

    for caps in data.values():
        [captions.append(cap) for cap in caps]
        return captions


def create_tokenizer(captions):
    tokenizer = Tokenizer()
    tokenizer.fit_on_texts(captions)
    return tokenizer

def longest_caption(captions):
    return max(len(cap.split()) for cap in captions)

In [12]:
captions = fetch_captions(train_tokens)
tokenizer = create_tokenizer(caption)
vocab_size = len(tokenizer.word_index) + 1
max_cap = longest_caption(captions)

In [13]:
def build_sequence(tokenizer,max_cap,vocab_size,feature,caption):
    input_1,input_2,output = [],[] , []

    for cap in captions:
        seq =tokenizer.texts_to_sequences([cap])[0]
        for indx in range(len(seq)):
            in_seq = seq[:indx]
            in_seq = pad_sequences([in_seq],maxlen= max_cap,padding ='post')[0]
            output_w = seq[indx]
            output_w = to_categorical([output_w],num_classes = vocab_size)[0]

            input_1.append(feature)
            input_2.append(in_seq)
            output.append(output_w)

    return np.array(input_1),np.array(input_2),np.array(output)



In [ ]:
def data_generator(tokenizer,features,data,max_cap,vocab_size):
    while True:
        for img,captions in data.items():
            if img in features:
                f=features[img][0]
                input_img,input_seq,output_word = build_sequence(tokenizer,max_cap,vocab_size,f,captions)

                yield [[input_img,input_seq],output_word]

In [14]:
[[in_img,in_seq],out_w] = next(data_generator(tokenizer,features,train_tokens,max_cap,vocab_size))

In [16]:
in_img.shape,in_seq.shape,out_w.shape

((0,), (0,), (0,))

In [ ]:
def build_model(num_features,longest_caption,output_size):
    # CNN model - images
    inputs_img = Input(shape=(num_features,))
    cnn_layer1 = Dropout(0.5)(inputs_img)
    cnn_layer2 = Dense(256,activation = 'relu')(cnn_layer1)


    # LSTM MODEL - TEXT -
    inputs_seq = Input (shape =(longest caption,))
    lstm_layer1 = Embedding(output_size,265,mask_zero= True)
    lstm_layer2 = Dropout(0.5)(lstm_layer1)
    lstm_layer3 = LSTM(256)(lstm_layer2)


    # MERGING :
    merged_model = add([cnn_layer2,lstm_layer3])
    final_model = Dense(256,activation ='relu')(merged_model)
    output = Dense(output_size , activation= 'softmax')(final_model)
    model = Model(inputs=[inputs_img,inputs_seq],outputs = output)
    model.compile(loss = 'categorical_crossentropy',optimizer = 'adam')
    return model

In [ ]:
model = build_model(featureSize,max_cap,vocab_size)